## Blockchain functionality 
Block linking process will have few tasks such as clubbing all the information to create a structure, calculating the hash of the block and appending it to the blockchain. Let's break down each of these functionalities into blockchain methods. 

In [11]:
# -*- coding: utf-8 -*-
import json

from Crypto.Hash import SHA256
from datetime import datetime


class Block(object):
    """A class representing the block for the blockchain"""

    def __init__(self, index, previous_hash, timestamp, data, hash):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.hash = hash






Above code snippet has python class called Block which has all the basic attributes to represent a block. Usually a block will contain both header and a body where header part will have metadata about the block. But, above example doesn’t distinguish between the header and body. A typical blockchain application such as bitcoin will have a huge set of data such as transactions, but we will consider data as just string type. 

In [12]:
class Blockchain(object):
    """A class representing list of blocks"""

    def __init__(self):

        self._chain = [self.get_genesis_block()]
        self.timestamp = datetime.now().strftime("%s")

Above class is a collection of class methods to create a valid blockchain by using hash function. Constructor of the Blockchain will initialize a chain by appending genesis block, which is the first block of the blockchain which doesn't have any reference to previous block. 

In [13]:
def get_genesis_block(self):
        """creates first block of the chain"""

        return Block(0, "0", 1465154705, "my genesis block!!",
                     "816534932c2b7154836da6afc367695e6337db8a921823784c14378abed4f7d7")


A genesis block is hardcoded block which will be appended to the beginning of the blockchain. It is created with all static contents. Above genesis block has a hardcoded hash value which is created using SHA256 as follows.

SHA256.new(data=(str(0) + "0"+ str(1465154705) +"my genesis block!!").encode()).hexdigest() 

In [14]:
    def calculate_hash(self, index, previous_hash, timestamp, data):
        """calculates SHA256 hash value"""

        hash_object = SHA256.new(data=(str(index) + previous_hash + str(timestamp) + data).encode())
        return hash_object.hexdigest()


calculate_hash is a crucial method of the blockchain as this method creates a hash value which will bind all the blocks together. SHA256 hash value is created by using PyCryptodome package as shown in the chapter 2. This method will concatenate block index, hash value of previous block, timestamp and data to create a string to be hashed. SHA256 hash function will generate a digest which will be the hash value of that block. 

In [15]:
    def get_latest_block(self):
        """gets the last block from the blockchain"""

        try:
            return self._chain[-1]
        except IndexError as e:
            return None

This function identifies the last block which is appended to the chain which is required while creating each block to find the hash value of previous block. 

In [16]:
    def create_block(self, block_data):
        """creates a new block with the given block data"""

        previous_block = self.get_latest_block()
        next_index = previous_block.index + 1
        next_timestamp = self.timestamp
        next_hash = self.calculate_hash(next_index, previous_block.hash, next_timestamp, block_data)
        return Block(next_index, previous_block.hash, next_timestamp, block_data, next_hash)


This function will build a block by constructing all the required attributes to create a Block object. It will also calculate the hash value for the current block. A new Block object consisting of the block structure will be finally created. 

In [17]:
    @property
    def chain(self):
        """created a dict containing list of block objects to view"""

        return self.dict(self._chain)

    def dict(self, chain):
        """converts list of block objects to dictionary"""

        return json.loads(json.dumps(chain, default=lambda o: o.__dict__))

    def reset(self):
        """resets the blockchain blocks except genesis block"""

        self._chain = [self._chain[0]]

    def add_block(self, data):
        """appends a new block to the blockchain"""

        self._chain.append(self.create_block(data))

All the above functions are used to add blocks, reset and read the blocks of the blockchain. Method add_block and the attribute chain are the only required class members that needs to be exposed to the user. 

## Creating a blockchain 

Now that we have defined all the required functionalities of a simple blockchain linker, let's emulate a blockchain linker by creating few blocks and adding them to the blockchain. 

The below code snippet creates a Blockchain object and adds three blocks to the blockchain along with an existing genesis block. This operation is performed again after resetting the blockchain. The important observation here is that both the output of new_chain.chain would produce same list of blocks containing same block hashes as below. This is due to the fact that all the attributes contributing to the hash value creation are same during both the execution, and hash function always produces the same hash value if the fed input is the same. The time stamp is intentionally kept constant for all the blocks to showcase the feature of the hash function. 

In [18]:
new_chain = Blockchain()
new_chain.add_block(data="modified first block data")
new_chain.add_block(data="second block data")
new_chain.add_block(data="third block data")

print(json.dumps(new_chain.chain))

AttributeError: 'Blockchain' object has no attribute 'get_genesis_block'

In [1]:
# example of iterating a nonce in a hashing algorithm's input

from __future__ import print_function
from Crypto.Hash import SHA256

text = "I am Satoshi Nakamoto"

# iterate nonce from 0 to 19
for nonce in range(20):

    # add the nonce to the end of the text
    input_data = text + str(nonce)

    # calculate the SHA-256 hash of the input (text+nonce)
    hash_data = SHA256.new(input_data.encode()).hexdigest()

    # show the input and hash result
    print((input_data + '=>' + hash_data)[:64] + "...")


I am Satoshi Nakamoto0=>a80a81401765c8eddee25df36728d732acb6d135...
I am Satoshi Nakamoto1=>f7bc9a6304a4647bb41241a677b5345fe3cd30db...
I am Satoshi Nakamoto2=>ea758a8134b115298a1583ffb80ae62939a2d086...
I am Satoshi Nakamoto3=>bfa9779618ff072c903d773de30c99bd6e2fd70b...
I am Satoshi Nakamoto4=>bce8564de9a83c18c31944a66bde992ff1a77513...
I am Satoshi Nakamoto5=>eb362c3cf3479be0a97a20163589038e4dbead49...
I am Satoshi Nakamoto6=>4a2fd48e3be420d0d28e202360cfbaba410bedde...
I am Satoshi Nakamoto7=>790b5a1349a5f2b909bf74d0d166b17a333c7fd8...
I am Satoshi Nakamoto8=>702c45e5b15aa54b625d68dd947f1597b1fa571d...
I am Satoshi Nakamoto9=>7007cf7dd40f5e933cd89fff5b791ff0614d9c60...
I am Satoshi Nakamoto10=>c2f38c81992f4614206a21537bd634af7178964...
I am Satoshi Nakamoto11=>7045da6ed8a914690f087690e1e8d662cf9e56f...
I am Satoshi Nakamoto12=>60f01db30c1a0d4cbce2b4b22e88b9b93f58f10...
I am Satoshi Nakamoto13=>0ebc56d59a34f5082aaef3d66b37a661696c2b6...
I am Satoshi Nakamoto14=>27ead1ca85da66981fd9da0